In [2]:
import spacy
from medspacy.context import ConTextComponent
from medspacy.visualization import visualize_ent
from scispacy.linking import EntityLinker
nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
nlp.add_pipe("medspacy_context")

doc = nlp("Alterations in the hypocretin receptor 2 and preprohypocretin genes produce narcolepsy in some animals.")
visualize_ent(doc)

/data/anaconda3/envs/scispacy/lib/python3.7/site-packages/requests/models.py:176: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(hook, collections.Callable):
/data/anaconda3/envs/scispacy/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/data/anaconda3/envs/scispacy/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-

In [3]:
# Let's look at a random entity!
entity = doc.ents[0]

print("Name: ", entity)

# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])

Name:  Alterations
CUI: C1515926, Name: Alteration
Definition: A change resulting in something that is different from the original.
TUI(s): T078
Aliases: (total: 1): 
	 alteration
CUI: C0013378, Name: Dysgeusia
Definition: A condition characterized by alterations of the sense of taste which may range from mild to severe, including gross distortions of taste quality.
TUI(s): T033
Aliases (abbreviated, total: 35): 
	 TASTE PERVERSION OF, Gustatory Alteration, abnormalities taste, Dysgeusia, taste disturbance, taste alteration, dysgeusias, TASTE ALTERATION, disturbances taste, Adverse taste perception
CUI: C0026882, Name: Mutation
Definition: Any detectable and heritable change in the genetic material that causes a change in the GENOTYPE and which is transmitted to daughter cells and to succeeding generations.
TUI(s): T045
Aliases (abbreviated, total: 14): 
	 Mutation, mutations, alterations genetic, genetic mutation, Genetic Alteration, mutation (genetics), genetic mutations, Mutations, 

In [56]:

text = """
No findings of Dipalmitoyl Phosphatidylcholine in pulmonary specimen.
"""
# text = "Alterations in the hypocretin receptor 2 and preprohypocretin genes produce narcolepsy in some animals."
doc = nlp(text)
visualize_ent(doc)

/data/anaconda3/envs/scispacy/lib/python3.6/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/data/anaconda3/envs/scispacy/lib/python3.6/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [5]:
doc.ents[0]._.is_negated

False

In [ ]:
# Run this when it's first time to build the pym.splite3
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *

default_world.set_backend(filename = "/data/joe/pym.sqlite3")
import_umls("/data/joe/umls-2022AA-full.zip", terminologies = ["SNOMEDCT_US", "CUI"])
default_world.save()

In [2]:
# Run this when reload the pysqlite3
from owlready2.pymedtermino2 import umls
umls.default_world.set_backend(filename = "/data/joe/pym.sqlite3")

In [3]:
from owlready2 import *
PYM = get_ontology("http://PYM/").load()
SNOMEDCT_US = PYM["SNOMEDCT_US"]
CUI = PYM["CUI"]

In [4]:
SNOMEDCT_US[302509004] >> CUI

Concepts([
  CUI["C1281570"] # Entire heart
])

In [14]:
CUI["C0003062"] >> SNOMEDCT_US

Concepts([
  SNOMEDCT_US["387961004"] # Kingdom Animalia
])